In [ ]:
class MPPIRacetrack:
    def __init__(
        self,
        static_map,
        motion_model=motion_models.Unicycle(),
    ):
        """ Your implementation here """
        self.motion_model = motion_model
        self.static_map = static_map
        self.motion_model = motion_model

        self.v_max = 1.0        # Linear Velocity max
        self.w_min = -2*np.pi
        self.w_max = 2*np.pi
        self.H = 20              # Horizon Length
        self.dt = 0.1           # Time Steps
        self.N = 50            # Number of Rollouts
        self.sig = 0.785        # Deviation for Noise
        self.lmda = 0.1

        self.waypoints = np.array([[-2, -2], [-3, 1], [-1.25, 3.75],
                                   [0.5, 4.65], [3.25, 3], [4, 1], [3,-2]])
        
        # raise NotImplementedError
    
    def get_nearest_goal(self, current_state):
        min_distance = float('inf')  # Initialize with a very large value
        nearest_index = -1           # Default to -1 if no valid waypoint is found
        for i, waypoint in enumerate(self.waypoints):
            dx = waypoint[0] - current_state[0]
            dy = waypoint[1] - current_state[1]
            distance = np.linalg.norm([dx, dy])
            theta_g = np.arctan2(dy, dx)
            dtheta = theta_g - current_state[2]
            dtheta = np.arctan2(np.sin(dtheta), np.cos(dtheta))
            if abs(dtheta) <= np.radians(90):  
                if distance < min_distance:
                    min_distance = distance
                    nearest_index = i

        return nearest_index
    
    def get_nominal_seq(self, initial_state, goal_pos):
        x, y, theta = initial_state
        nominal_seq = []
        w_min, w_max = -2*np.pi, 2*np.pi

        for _ in range(self.H):
            angle_to_goal = np.arctan2(goal_pos[1] - y, goal_pos[0] - x)
            angle_diff = angle_to_goal - theta
            angle_diff = (angle_diff + np.pi) % (2 * np.pi) - np.pi  # Normalize to [-π, π]
            w = np.clip(angle_diff / self.dt, w_min, w_max)

            distance_to_goal = np.linalg.norm([goal_pos[0] - x, goal_pos[1] - y])
            v = self.v_max if distance_to_goal > self.v_max * self.dt else distance_to_goal / self.dt

            nominal_seq.append([v, w])
            x += v * np.cos(theta) * self.dt
            y += v * np.sin(theta) * self.dt
            theta += w * self.dt
        
        return nominal_seq
    
    # Cost Function
    def cost_function(self, initial_state, goal_pos, seq):
        x, y, theta = initial_state
        distances = []
        ctrl_eff = []

        for v, w in seq:
            x += v * np.cos(theta) * self.dt
            y += v * np.sin(theta) * self.dt
            theta += w * self.dt
            distances.append(np.linalg.norm([x - goal_pos[0], y - goal_pos[1]]))
            ctrl_eff.append(v**2 + w**2)
        
        dist_cost = np.mean(distances)
        ctrl_cost = np.mean(ctrl_eff)
        total_cost = 0.99*dist_cost + 0.01*ctrl_cost
        # print(total_cost)

        return total_cost
    
    def update_control(self, nominal_seq, rollouts, weights):
        du = rollouts - nominal_seq
        seq = np.zeros_like(nominal_seq)
       
        for col in range(len(nominal_seq)):
            seq[col] = np.sum(weights[:, None] * du[:, col], axis=0) / np.sum(weights)

        final_seq = nominal_seq + seq
        final_seq[:, 0] = np.clip(final_seq[:, 0], 0, 1.0)  # Clip v between 0 and 1.0
        final_seq[:, 1] = np.clip(final_seq[:, 1], -2 * np.pi, 2 * np.pi)  # Clip w between -2π and 2π

        return final_seq

    def get_action(self, initial_state: np.array) -> np.array:
        """ Your implementation here """
        goal_pos = self.waypoints[self.get_nearest_goal(initial_state)]
        nominal_seq = self.get_nominal_seq(initial_state, goal_pos)

        rollouts = np.array([[[np.clip(v + np.random.normal(0, self.sig), 0, self.v_max),  
                               np.clip(w + np.random.normal(0, self.sig), -2 * np.pi, 2 * np.pi)] for v, w in nominal_seq] 
                               for _ in range(self.N)])
        scores = np.array([self.cost_function(initial_state, goal_pos, seq) for seq in rollouts])
        weights = np.exp(-1 * scores / self.lmda)
        ctrl_seq = self.update_control(nominal_seq, rollouts, weights)
        action = np.array([0.1, np.radians(5)/self.dt])
        action = np.array(ctrl_seq[0])
        # raise NotImplementedError
        return action

In [9]:
for i in range(14):
    print(f"{0.03*(i+7)}")

0.21
0.24
0.27
0.3
0.32999999999999996
0.36
0.39
0.42
0.44999999999999996
0.48
0.51
0.54
0.57
0.6
